<a href="https://colab.research.google.com/github/Nicolas-Ryd/Google-Colab-identificaci-n-de-imagenes-Gato-y-Perro/blob/main/Gato_Perro_Versi%C3%B3n_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fase 1. Fases de inferencia
 En esta primera fase, prepararemos el entorno para el funcionamiento del resto del código y nuestra inteligencia artificial detectorra de perros y gatos, definiendo ambas categorias para su posterior clasificación.

In [ ]:
import os
# 1. Activamos el modo legacy (Keras 2)
os.environ['TF_USE_LEGACY_KERAS'] = '1'

from tensorflow import keras

#Carga el modelo
mi_modelo = keras.models.load_model("/content/drive/MyDrive/keras_model.h5", compile=False)

# Carga las etiquetas de las clases
nombre_clases = open("/content/drive/MyDrive/labels.txt", "r").readlines()

print(nombre_clases)


['0 Perro\n', '1 Gato\n']


#Fase 2: Adquirir Datos Nuevos
Ahora lo que introduciremos, será la siguiente linea de código para poder subir a google drive las imagenes de prueba que deseamos que el modelo clasifique.

In [ ]:
from PIL import Image
imagen= Image.open("/content/drive/MyDrive/Ejercicios perras y gatas/cats_dogs_light/test/cat.9828.jpg").convert("RGB")

#Fase 3: Pre-procesamiento
En esta fase, la imagen debe transformarse para ser idéntica a las de entrenamiento por lo que debemos pasar las imagenes de clasificación al mismo tamaño.

In [ ]:
from PIL import Image, ImageOps
import numpy as np
size = (224, 224)
imagen = ImageOps.fit(imagen, size, Image.Resampling.LANCZOS)
imagen_array = np.asarray(imagen)
normalizada_imagen_array = (imagen_array.astype(np.float32) / 127.5) - 1

#Fase 4: Predicción
En esta fase, crearemos un lote para que sea analizado por la IA

In [ ]:
lote_imagenes = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
lote_imagenes[0] = normalizada_imagen_array
resultados= mi_modelo.predict(lote_imagenes)

1/1 [==============================] - 3s 3s/step


# Fase 5: Post-procesamiento
En esta fase se busca el índice del número más alto en el array devuelto por el modelo (en [0.98, 0.02] sería el índice 0). Se usa el índice para buscar en la lista de etiquetas (labels.txt) y se muestra la etiqueta correspondiente (ej. "Perro").


In [ ]:
indice = np.argmax(resultados[0])
etiqueta = nombre_clases[indice]
print("La imagen es de clase: ", etiqueta)
probabilidad = resultados[0][indice]
print("Con una probabilidad de: ", probabilidad)

La imagen es de clase:  1 Gato

Con una probabilidad de:  0.9999993


#Paso Intermedio
###Definir predecir_imagen
En esta sección de nuestro programa, definiremos la función para poder usarla más adelante. Primero cargaremos y redimensionaremos la imagen y bpasamos a array Numpy, cargamosndoo el lote de imagenes en el proceso. A continuación realizamos la predicción



In [ ]:
def predecir_imagen(ruta_imagen):
    imagen = Image.open(ruta_imagen).convert("RGB")
    size = (224, 224)
    imagen = ImageOps.fit(imagen, size, Image.Resampling.LANCZOS)


    imagen_array = np.asarray(imagen)
    normalizada_imagen_array = (imagen_array.astype(np.float32) / 127.5) - 1
    lote_imagenes = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    lote_imagenes[0] = normalizada_imagen_array

    resultados = mi_modelo.predict(lote_imagenes, verbose=0)
    indice = np.argmax(resultados[0])
    etiqueta = nombre_clases[indice]
    etiqueta_limpia = etiqueta.split(' ', 1)[1].strip().lower()

    probabilidad = resultados[0][indice]

    return etiqueta_limpia, probabilidad

print("La función 'predecir_imagen' ha sido refactorizada.")

La función 'predecir_imagen' ha sido refactorizada.


#Fase 6: Preparación para lectura de carpetas
Finalmente, creamos nuevas variables para indicar los differentes resultados qu nos puede darusando la definición de predecir_imagen, obtenemos los porcentajes y las imagenes falladas

In [ ]:
lista_archivos = os.listdir("/content/drive/MyDrive/Ejercicios perras y gatas/cats_dogs_light/test")
total_predicciones = 0
aciertos = 0
media_probabilidad_aciertos = 0.0
# predicciones incorrectas contendrá una lista con los nombres de las imágenes mal clasificadas
predicciones_incorrectas = []

for nombre_archivo in lista_archivos:
    if "cat" in nombre_archivo:
        etiqueta_esperada = "gato"
    elif "dog" in nombre_archivo:
        etiqueta_esperada = "perro"
    else:
        continue  # Saltar archivos que no sean de gatos o perros

    total_predicciones += 1

    ruta_imagen = os.path.join("/content/drive/MyDrive/Ejercicios perras y gatas/cats_dogs_light/test", nombre_archivo)

    # Aquí obtendriamos la predicción del modelo llamando a una función que
    # implemente las fases de inferencia
    etiqueta_predicha, probabilidad = predecir_imagen(ruta_imagen)

    if etiqueta_predicha == etiqueta_esperada:
        aciertos += 1
        media_probabilidad_aciertos += probabilidad
    else:
        info_error= {"archivo": nombre_archivo, "prediccion": etiqueta_predicha,
             "probabilidad": probabilidad}
        predicciones_incorrectas.append(info_error)


# Fase 7: Cálculo de métricas y generación del informe
Finalmente, vamos a calcular las métricas de rendimiento del modelo y generar un informe con los resultados obtenidos. Para ello, usaremos los contadores inicializados en la sección anterior y generaremos un informe que publicaremos en un repositorio de GitHub, el cual nos servirá como portafolio de proyectos.

In [ ]:
precision = aciertos / total_predicciones if total_predicciones > 0 else 0
media_probabilidad_aciertos /= aciertos if aciertos > 0 else 1
informe = f"""
Informe de Evaluación del Modelo

Total de predicciones: {total_predicciones}
Aciertos: {aciertos}
Precisión: {precision:.2%}
Probabilidad media de aciertos: {media_probabilidad_aciertos:.2%}

Predicciones incorrectas:
"""

for error in predicciones_incorrectas:
    informe += f"Archivo: {error['archivo']}, Prediccion: {error['prediccion']}, Probabilidad: {error['probabilidad']:.2%}\n"

print(informe)


Informe de Evaluación del Modelo

Total de predicciones: 400
Aciertos: 390
Precisión: 97.50%
Probabilidad media de aciertos: 99.30%

Predicciones incorrectas:
Archivo: dog.9808.jpg, Prediccion: gato, Probabilidad: 84.00%
Archivo: dog.9868.jpg, Prediccion: gato, Probabilidad: 93.13%
Archivo: dog.9911.jpg, Prediccion: gato, Probabilidad: 99.96%
Archivo: dog.9935.jpg, Prediccion: gato, Probabilidad: 99.96%
Archivo: cat.9836.jpg, Prediccion: perro, Probabilidad: 98.81%
Archivo: cat.9874.jpg, Prediccion: perro, Probabilidad: 84.85%
Archivo: cat.9882.jpg, Prediccion: perro, Probabilidad: 98.40%
Archivo: cat.9954.jpg, Prediccion: perro, Probabilidad: 79.82%
Archivo: cat.9960.jpg, Prediccion: perro, Probabilidad: 91.54%
Archivo: cat.9947.jpg, Prediccion: perro, Probabilidad: 92.40%

